In [5]:
from google import genai
from google.genai import types
import time
from dotenv import load_dotenv
import os
import json

load_dotenv()

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

In [12]:
with open("/Users/aniruddha/Documents/Code/Metatuning/ques.json", "rb") as f:
  data = json.load(f)

data = data[0:50]

In [14]:
def prepare_question(question_data):
    question = question_data["question"]
    choices_list = []
    answer = None
    question_type = question_data["question_type"]
    if 'choices' in question_data:
        choices = question_data["choices"]
        for idx,choice in enumerate(choices):
            choices_list.append(choice['choice'])
            choice_answer = choice['answer']
            if choice_answer == 'correct':
                answer = f"option {idx+1} : "+ choice['choice']
    else:
        choices_list = None
        answer = question_data["answer"]
    
    if choices_list:
        prompt = f'Question : {question}\n\n the {len(choices_list)} choices are : {choices_list}'
    else:
        prompt = f'Question : {question}'

    return question, question_type,answer,prompt
    

In [ ]:
with open("/Users/aniruddha/Documents/Code/Metatuning/gemini-wo-metatuning-clever.json", "rb") as f:
     output = json.load(f)

for idx, question_data in enumerate(data):
    questions = question_data["questions"]
    video_filename = question_data["video_filename"]
    print("*"*100)
    print(f"doing video : {video_filename}")
    myfile = client.files.upload(file=f"/Users/aniruddha/Documents/Code/Metatuning/video_train/video_00000-01000/{video_filename}")
    while myfile.state.name == "PROCESSING":
            print("processing video...")
            time.sleep(5)
            myfile = client.files.get(name=myfile.name)
    print(f"{myfile=}")
    if f"{video_filename}" not in output:
      output[f"{video_filename}"] = {}
    for idx, question_data in enumerate(questions):
        print(f"doing question : {idx}")
        question, question_type,answer,prompt = prepare_question(question_data)
        result = client.models.generate_content(
            model="gemini-2.0-flash", contents=[myfile,prompt], 
            config=types.GenerateContentConfig(
              system_instruction=
                [
                  'You are a visual question answering agent. Answer the question based on the video.',
                  'Keep your answer as short as possible. Keep it to a single word when possible. If the answer is blue just say blue and nothing else.'
                ]
          )
        )
        output[f"{video_filename}"][f"question_{idx}"] = {"question":question,
                                                      "question_type" : question_type,
                                                      "prompt":prompt,
                                                      "answer":answer,
                                                      "llm_answer":result.text}

    with open("/Users/aniruddha/Documents/Code/Metatuning/gemini-wo-metatuning-clever.json", "w") as f:
        json.dump(output, f, indent=4)

## Metatuning

In [23]:
with open("/Users/aniruddha/Documents/Code/Metatuning/ques.json", "rb") as f:
  data = json.load(f)

data = data[43:50]

In [24]:
def prepare_question(question_data):
    question = question_data["question"]
    choices_list = []
    answer = None
    question_type = question_data["question_type"]
    if 'choices' in question_data:
        choices = question_data["choices"]
        for idx,choice in enumerate(choices):
            choices_list.append(choice['choice'])
            choice_answer = choice['answer']
            if choice_answer == 'correct':
                answer = f"option {idx+1} : "+ choice['choice']
    else:
        choices_list = None
        answer = question_data["answer"]
    
    if choices_list:
        prompt = f'Question : {question}\n\n the {len(choices_list)} choices are : {choices_list}'
    else:
        prompt = f'Question : {question}'

    return question, question_type,answer,prompt
    

In [25]:
file_path = "/Users/aniruddha/Documents/Code/Metatuning/gemini-with-metatuning-clever.json"
with open(file_path, "rb") as f:
     output = json.load(f)

for idx, question_data in enumerate(data):
    questions = question_data["questions"]
    video_filename = question_data["video_filename"]
    print("*"*100)
    print(f"doing video : {video_filename}")
    myfile = client.files.upload(file=f"/Users/aniruddha/Documents/Code/Metatuning/video_train/video_00000-01000/{video_filename}")
    while myfile.state.name == "PROCESSING":
            print("processing video...")
            time.sleep(5)
            myfile = client.files.get(name=myfile.name)
    print(f"{myfile=}")
    if f"{video_filename}" not in output:
      output[f"{video_filename}"] = {}

    #prepare examples
    examples = []
    for idx, question_data in enumerate(questions):
        question, question_type,answer,prompt = prepare_question(question_data)
        examples.append(f"example {idx+1} : prompt : {prompt}\n\n answer : {answer} \n\n")
        if idx >3:
            break
    print("examples prepared")
    for idx, question_data in enumerate(questions):
        if idx < 3:
            continue
        print(f"doing question : {idx}")
        question, question_type,answer,prompt = prepare_question(question_data)
        result = client.models.generate_content(
            model="gemini-2.0-flash", contents=[myfile,prompt], 
            config=types.GenerateContentConfig(
              system_instruction=
                [
                  'You are a visual question answering agent. Answer the question based on the video.',
                  'Keep your answer as short as possible. Keep it to a single word when possible. If the answer is blue just say blue and nothing else.',
                  "Here are few examples of how to answer the question given a prompt for the current video : " + "\n".join(examples)
                ]
          )
        )
        output[f"{video_filename}"][f"question_{idx}"] = {"question":question,
                                                      "question_type" : question_type,
                                                      "prompt":prompt,
                                                      "answer":answer,
                                                      "llm_answer":result.text}

    with open(file_path, "w") as f:
        json.dump(output, f, indent=4)

****************************************************************************************************
doing video : video_00043.mp4
processing video...
myfile=File(name='files/9yutu4m6t4kj', display_name=None, mime_type='video/mp4', size_bytes=1795426, create_time=datetime.datetime(2025, 4, 16, 13, 46, 7, 59355, tzinfo=TzInfo(UTC)), expiration_time=datetime.datetime(2025, 4, 18, 13, 46, 7, 14600, tzinfo=TzInfo(UTC)), update_time=datetime.datetime(2025, 4, 16, 13, 46, 8, 173177, tzinfo=TzInfo(UTC)), sha256_hash='ZDlhNGJhNTcyOGY2NWRiYTYyODM3YTc0MmIxMmYzYjQzOWYyNGU0MDMxN2MwYzQ3NmZiOGE0OGE2NTNjZTIxZA==', uri='https://generativelanguage.googleapis.com/v1beta/files/9yutu4m6t4kj', download_uri=None, state=<FileState.ACTIVE: 'ACTIVE'>, source=<FileSource.UPLOADED: 'UPLOADED'>, video_metadata={'videoDuration': '5s'}, error=None)
examples prepared
doing question : 3
doing question : 4
doing question : 5
doing question : 6
doing question : 7
doing question : 8
doing question : 9
doing question : 1

## Metatuning with examples in user prompt and not system prompt

In [32]:
with open("/Users/aniruddha/Documents/Code/Metatuning/ques.json", "rb") as f:
  data = json.load(f)

data = data[37:50]

In [33]:
def prepare_question(question_data):
    question = question_data["question"]
    choices_list = []
    answer = None
    question_type = question_data["question_type"]
    if 'choices' in question_data:
        choices = question_data["choices"]
        for idx,choice in enumerate(choices):
            choices_list.append(choice['choice'])
            choice_answer = choice['answer']
            if choice_answer == 'correct':
                answer = f"option {idx+1} : "+ choice['choice']
    else:
        choices_list = None
        answer = question_data["answer"]
    
    if choices_list:
        prompt = f'Question : {question}\n\n the {len(choices_list)} choices are : {choices_list}'
    else:
        prompt = f'Question : {question}'

    return question, question_type,answer,prompt
    

In [34]:
file_path = "/Users/aniruddha/Documents/Code/Metatuning/gemini-with-metatuning-clever-user-prompt.json"
with open(file_path, "rb") as f:
     output = json.load(f)

for idx, question_data in enumerate(data):
    questions = question_data["questions"]
    video_filename = question_data["video_filename"]
    print("*"*100)
    print(f"doing video : {video_filename}")
    myfile = client.files.upload(file=f"/Users/aniruddha/Documents/Code/Metatuning/video_train/video_00000-01000/{video_filename}")
    while myfile.state.name == "PROCESSING":
            print("processing video...")
            time.sleep(5)
            myfile = client.files.get(name=myfile.name)
    print(f"{myfile=}")
    if f"{video_filename}" not in output:
      output[f"{video_filename}"] = {}

    #prepare examples
    examples = []
    for idx, question_data in enumerate(questions):
        question, question_type,answer,prompt = prepare_question(question_data)
        examples.append(f"example {idx+1} : prompt : {prompt}\n\n answer : {answer} \n\n")
        if idx >3:
            break
    print("examples prepared")
    for idx, question_data in enumerate(questions):
        if idx < 3:
            continue
        print(f"doing question : {idx}")
        question, question_type,answer,prompt = prepare_question(question_data)
        result = client.models.generate_content(
            model="gemini-2.0-flash", contents=[myfile,prompt], 
            config=types.GenerateContentConfig(
              system_instruction=
                [
                  'You are a visual question answering agent. Answer the question based on the video.',
                  'Keep your answer as short as possible. Keep it to a single word when possible. If the answer is blue just say blue and nothing else.',
                  "Here are few examples of how to answer the question given a prompt for the current video"
                ]
          )
        )
        output[f"{video_filename}"][f"question_{idx}"] = {"question":question,
                                                      "question_type" : question_type,
                                                      "prompt":prompt + "\n\n" + "\n".join(examples),
                                                      "answer":answer,
                                                      "llm_answer":result.text}

    with open(file_path, "w") as f:
        json.dump(output, f, indent=4)

****************************************************************************************************
doing video : video_00037.mp4
processing video...
myfile=File(name='files/qw4zoho9ogbj', display_name=None, mime_type='video/mp4', size_bytes=1371026, create_time=datetime.datetime(2025, 4, 19, 12, 56, 49, 307228, tzinfo=TzInfo(UTC)), expiration_time=datetime.datetime(2025, 4, 21, 12, 56, 49, 269872, tzinfo=TzInfo(UTC)), update_time=datetime.datetime(2025, 4, 19, 12, 56, 50, 291189, tzinfo=TzInfo(UTC)), sha256_hash='NjFhZTYzYWY3ZjFmZWFiZmZjZWY4NTVlMmM5YjA3NWEyYzk1NjEyNjRlOGIyZDg0YjBkODEyMzMwN2I4MGI0OQ==', uri='https://generativelanguage.googleapis.com/v1beta/files/qw4zoho9ogbj', download_uri=None, state=<FileState.ACTIVE: 'ACTIVE'>, source=<FileSource.UPLOADED: 'UPLOADED'>, video_metadata={'videoDuration': '5s'}, error=None)
examples prepared
doing question : 3
doing question : 4
doing question : 5
doing question : 6
doing question : 7
doing question : 8
doing question : 9
doing questio